In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [ ]:
import keras
print(keras.__version__)

2.15.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/RecipeFusion/fusionrecipe.csv')


In [ ]:
data.head()

,name,image_url,description,cuisine,course,calories & servings,diet,prep_time,cook_time,ingredients,instruction,image_id
0,Indian Fry Bread,https://media.istockphoto.com/id/691985214/pho...,Fried bread is a slice of bread that has been ...,Indian,Appetizer,682 & 4,Veg,45 minutes,20 minutes,"4 cups flour, 2 tbsp. baking powder, 1 tsp. sa...","Mix flour, baking powder and salt. Slowly add ...",0
1,Baked Indian Pudding,https://s3.amazonaws.com/grecipes/public/pictu...,Delicious baked pudding with maple syrup. Grea...,Indian,Dessert,581 & 4,Veg,25 minutes,60 minutes,"A½ cup cornmeal, 4 cups hot milk, 2 tablespoon...",Preheat Oven 325 degrees: In a double-broiler ...,1
2,Cabbage Pachadi (salad),https://s3.amazonaws.com/grecipes/public/pictu...,Cabbage Pachadi is a South Indian salad made w...,Indian,Side Dish,134 & 8,Veg,10 minutes,0 minutes,1/1 a medium cabbage (inner leaves only) finel...,"Place the cabbage, tomato, peanut powder, coco...",2
3,Shepuchi Bhaji,http://grecipes.s3.amazonaws.com/recipe_pictur...,Shepuchi Bhaji is a traditional Maharashtrian ...,Indian,Main Course,31 & 4,Veg,10 minutes,20 minutes,"1 bunch Dill leaves / Shepu leaves, 2 tbsp. Mo...",1.Soak moong daal for 30 minutes. 2.Chop Dill ...,3
4,Beetroot Cutlet,https://s3.amazonaws.com/grecipes/public/pictu...,"To make beetroot cutlets, grate boiled beetroo...",Indian,Appetizer,190 & 4,Veg,20 minutes,15 minutes,"Beetroot - 1 No. (small), Potato - 2 Nos. (med...",Pressure cook Potato and beetroot till 3 whist...,4


In [ ]:
print("Dataset info:")
print(data.info())


Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 1152 non-null   object
 1   image_url            1152 non-null   object
 2   description          1152 non-null   object
 3   cuisine              1152 non-null   object
 4   course               1151 non-null   object
 5   calories & servings  1152 non-null   object
 6   diet                 1152 non-null   object
 7   prep_time            1152 non-null   object
 8   cook_time            1152 non-null   object
 9   ingredients          1152 non-null   object
 10  instruction          1144 non-null   object
 11  image_id             1152 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 108.1+ KB
None


In [ ]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Download NLTK resources
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Convert any non-string data to string
data['instruction'] = data['instruction'].astype(str)

# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['instruction'])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in data['instruction']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = y.reshape(y.shape[0], 1)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Build the RNN model
rnn_model = Sequential()
rnn_model.add(Embedding(total_words, 200, input_length=max_sequence_length-1))
rnn_model.add(LSTM(200))
rnn_model.add(Dense(total_words, activation='softmax'))


In [ ]:
# Build the RNN model with increased complexity and regularization
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
rnn_model = Sequential()
rnn_model.add(Embedding(total_words, 200, input_length=max_sequence_length-1))
rnn_model.add(LSTM(200, return_sequences=True))
rnn_model.add(Dropout(0.2))  # Adding dropout for regularization
rnn_model.add(LSTM(100))  # Adding another LSTM layer
rnn_model.add(Dense(total_words, activation='softmax'))

In [ ]:
# Compile the model
rnn_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
rnn_model.fit(X_train, y_train, epochs=50, verbose=1, validation_split=0.1)

Epoch 1/50
3945/3945 [==============================] - 156s 39ms/step - loss: 0.9385 - accuracy: 0.7280 - val_loss: 2.9035 - val_accuracy: 0.5396
Epoch 2/50
3945/3945 [==============================] - 151s 38ms/step - loss: 0.9311 - accuracy: 0.7297 - val_loss: 2.9204 - val_accuracy: 0.5370
Epoch 3/50
3945/3945 [==============================] - 154s 39ms/step - loss: 0.9238 - accuracy: 0.7307 - val_loss: 2.9121 - val_accuracy: 0.5386
Epoch 4/50
3945/3945 [==============================] - 150s 38ms/step - loss: 0.9180 - accuracy: 0.7320 - val_loss: 2.9304 - val_accuracy: 0.5392
Epoch 5/50
3945/3945 [==============================] - 155s 39ms/step - loss: 0.9132 - accuracy: 0.7332 - val_loss: 2.9368 - val_accuracy: 0.5414
Epoch 6/50
3945/3945 [==============================] - 151s 38ms/step - loss: 0.9136 - accuracy: 0.7331 - val_loss: 2.9394 - val_accuracy: 0.5391
Epoch 7/50
3945/3945 [==============================] - 150s 38ms/step - loss: 0.9072 - accuracy: 0.7359 - val_loss: 2

In [ ]:
# Train the model
rnn_model.fit(X_train, y_train, epochs=30, verbose=1)


In [ ]:
rnn_model.save('/content/drive/MyDrive/RecipeFusion/fusionrecipenayawala_rnn.h5')

NameError: name 'rnn_model' is not defined

In [ ]:
 loss, accuracy = rnn_model.evaluate(X_test, y_test, verbose=0)

In [ ]:
 y_pred = [generate_next_word(model, sequence, tokenizer, max_sequence_length) for sequence in text_sequences]
    f1 = f1_score(y_test, y_pred, average='weighted')

    return loss, accuracy, f1

In [ ]:
loss, accuracy, f1 = evaluate_model(rnn_model, tokenizer, max_sequence_length, X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)
print("F1 Score:", f1)

In [ ]:
rnn_model.save('/content/drive/MyDrive/Recipefusion/fusionrecipe_rnn.keras')  # The file needs to end with the .keras extension

In [ ]:
rnn_model.summary()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Generate text sequences for the test set
def generate_text_sequences(model, tokenizer, max_sequence_length, X_test):
    text_sequences = []
    for i in range(len(X_test)):
        input_sequence = X_test[i]
        input_text = ' '.join([tokenizer.index_word[idx] for idx in input_sequence if idx != 0])
        text_sequences.append(input_text)
    return text_sequences


In [ ]:
def evaluate_model(model, tokenizer, max_sequence_length, X_test, y_test):
    # Generate text sequences for the test set
    text_sequences = generate_text_sequences(model, tokenizer, max_sequence_length, X_test)

    # Evaluate accuracy
    y_pred = []
    for sequence in text_sequences:
        # Generate the next word
        predicted_index = generate_next_word(model, sequence, tokenizer, max_sequence_length)
        y_pred.append(predicted_index)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred, average='weighted')

    return accuracy, f1

In [ ]:
# Function to generate the next word in a sequence
def generate_next_word(model, input_text, tokenizer, max_sequence_length):
    # Convert input text to sequence
    input_sequence = tokenizer.texts_to_sequences([input_text])[0]
    padded_sequence = pad_sequences([input_sequence], maxlen=max_sequence_length-1, padding='pre')
    # Predict next word
    prediction = model.predict(padded_sequence)
    # Get the index of the predicted word
    predicted_index = np.argmax(prediction)
    return predicted_index

In [ ]:
# Evaluate the model
accuracy, f1 = evaluate_model(rnn_model, tokenizer, max_sequence_length, X_test, y_test)
print("Accuracy:", accuracy)
print("F1 Score:", f1)

In [ ]:
import random
import numpy as np

# Function to generate text given a seed text
def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [ ]:

# Function to predict recipe instructions and ingredients based on the recipe name
def predict_recipe(recipe_name, model, max_sequence_length):
    recommended_recipe = generate_text(recipe_name, next_words=500, model=rnn_model, max_sequence_length=max_sequence_length)
    return recommended_recipe

# User input for the recipe name
user_recipe_name = input("Enter a recipe name: ")

# Generate recommendations based on the user's input
recommended_recipe = predict_recipe(user_recipe_name, rnn_model, max_sequence_length)
print("Recommended Recipe Instructions:")
print(recommended_recipe)


In [ ]:
import keras
model = keras.models.load_model('/content/drive/MyDrive/RecipeFusion/fusionrecipenayawala_rnn.h5')

In [ ]:
model.summary()

In [ ]:
import random
import numpy as np

# Function to generate text given a seed text
def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [ ]:

# Function to predict recipe instructions and ingredients based on the recipe name
def predict_recipe(recipe_name, model, max_sequence_length):
    recommended_recipe = generate_text(recipe_name, next_words=500, model=model, max_sequence_length=max_sequence_length)
    return recommended_recipe

# User input for the recipe name
user_recipe_name = input("Enter a recipe name: ")

# Generate recommendations based on the user's input
recommended_recipe = predict_recipe(user_recipe_name, model, max_sequence_length)
print("Recommended Recipe Instructions:")
print(recommended_recipe)
